In [ ]:
# liver tumour segmentation app

In [1]:
import gradio as gr
from PIL import Image
import numpy as np
from fastai.vision.all import *
from pathlib import Path

# Assuming your segmentation mask files are stored in a directory
mask_dir = Path("path/to/segmentation_masks")

def label_func(x):
    # Assuming x is the path to the input image
    # Extract the filename of the input image
    img_filename = x.name
    
    # Assuming the corresponding mask filename follows a certain pattern
    mask_filename = img_filename.replace(".jpg", "_mask.png")
    
    # Construct the path to the segmentation mask file
    mask_path = mask_dir / mask_filename
    
    # Return the path to the segmentation mask file
    return mask_path
    
from fastai.metrics import foreground_acc

# Define or import the custom foreground accuracy function
def cust_foreground_acc(inp, targ):
    return foreground_acc(inp=inp, targ=targ)

# Now, load the model
model_path = "liver_segmentation_resnet50.pkl"
learn0 = load_learner(model_path)


D:\ML\Lib\site-packages\torch\cuda\__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [2]:
def predict_segmentation(ct_scan):
    # Convert PIL image to numpy array
    ct_scan_np = np.array(ct_scan)
    
    # Prepare CT scan for prediction
    ct_scan_tensor = tensor(ct_scan_np).permute(2, 0, 1)
    ct_scan_tensor = (ct_scan_tensor.unsqueeze(0).float() / 255.0)
    
    # Get predictions
    preds, _ = learn0.get_preds(dl=[(ct_scan_tensor,)])
    predicted_mask = np.argmax(preds.squeeze().numpy(), axis=0)
    
    # Convert the predicted mask to PIL Image
    segmented_image = Image.fromarray(predicted_mask.astype(np.uint8) * 255)
    
    return segmented_image

# # Define the input component for uploading CT scan image
# ct_scan_input = gr.inputs.Image(shape=(512, 512), type='pil')

# # Define the output component for displaying segmented image
# segmentation_output = gr.outputs.Image(type='pil')

# Create the Gradio interface
gr.Interface(fn=predict_segmentation, inputs=["image"], outputs=["image"], 
              title='Liver Tumor Segmentation',
              description='Upload a CT scan image to segment liver tumors.').launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
